## Coinbase Trasaction Statement.
Contains information regarding trsaction performed on the base Coinbase platform (not Coinbase Pro/GDAX)
These include receipt or transfers of money or crypto, as well as any coinbase assisted trades/sales
### How to get:
- login to Coinbase.com
- Go to: https://accounts.coinbase.com/profile 
- Select "statements"
- Click on the "transactions" tab
- "Generate custom statement" with:
    - "all assets"
    - "all transactions"
    - Select the desired year

What we are interested in here are:
- BTC/ETH "Receive" - Receipt of new crypto (income) as an Acquisition - tho maybe also just a transfer into CB before a sale in cbrpo? One tip is that even-numbered values from an ext acct are probably from me.
- BTC/ETH "Withdrawl" - implicit CB-assisted sale as a Disposition
- BTC/ETH "Send" - maybe a payment (Disposition) or maybe just a transfer out of CB (nothing reportable)

Less directly interesting are:

BTC/ETH "Exchange Deposit" - Crypto being sent to CBPro/GDAX - almost certainly to be sold
USD "Exchange withdrawal" - almost always right after the BTC Exchange deposit - is the cash from the sale
USD "Withdrawal" - usually the same amount as the exchange withdrawal, and right after it: the money being transferred to a bank

Start by assuming "receive" is an acq, and "send' and an asset "withdrawl" are always dispositions

Aslo - just as an aside - the records in these TX statements are ordered newest to oldest... which is weird.


In [105]:
# allow import of local fifo-tool stuff
from typing import List
import os
import sys
sys.path.insert(0, os.path.abspath('../src'))

In [106]:
from typing import Dict
from datetime import datetime
import json
import numpy as np
import pandas as pd

from models.acquisition import Acquisition
from models.disposition import Disposition
from models.stash import Stash

In [107]:
def read_statement_csv(file_path):
    """Read a coinbase transaction statement and return a pandas dataframe.
        conversions done:
            'time' - parsed into a datestamp
            'trade id' - read as a string
    """
    dol = lambda x: float(x.replace('$','')) # money fields seem to have $ signs...
    date_flds = ['Timestamp']
    dollar_flds = {'Price at Transaction':dol,'Subtotal':dol,'Total (inclusive of fees and/or spread)':dol,'Fees and/or Spread':dol}
    forced_dtypes = {}
    df =  pd.read_csv(file_path, skiprows=3, parse_dates=date_flds, converters=dollar_flds)
    return df

In [108]:
# tx_type: Send, Receive, Withdrawal
def parse_transactions(data, asset, tx_type):
    type_mask = data["Transaction Type"]==tx_type
    asset_mask = data["Asset"]==asset
    return data[asset_mask & type_mask]    


In [109]:
def rec_to_disposition(rec, asset) -> 'Disposition':
    """ rec is a single file row as a dict keyed by column names """
    return Disposition(
        rec['Timestamp'].timestamp(),
        rec['Asset'] , # asset_type sold
        rec['Quantity Transacted'], # asset_amount
        rec['Price at Transaction'], # asset_price,
        rec['Fees and/or Spread'], # fees
        f"CB Id: {rec['ID']}", # reference
        rec['Notes']
    )

def rec_to_acquisition(rec, asset) -> 'Acquisition':
    return Acquisition(
        rec['Timestamp'].timestamp(),
        rec['Asset'], # asset_type bought/received
        rec['Quantity Transacted'], # asset_amount
        rec['Price at Transaction'], # asset_price,
        rec['Fees and/or Spread'], # fees
        f"Coinbase ID: {rec['ID']} - {rec['Notes']}" #comment
    )

In [110]:

def process_file( year: str, assets: List[str]) -> None:
    filebase = f'local_data/coinbase-txs-{year}'
    main_df = read_statement_csv(filebase+'.csv')

    for asset in assets:
        rcv_df =  parse_transactions(main_df, asset, 'Receive') # cryto receive *might* be an acquisition... or just a self-transfer
        rcv_recs = [r for r in rcv_df.to_dict(orient='index').values()] # converts DataFrame into a list of dicts
        send_df = parse_transactions(main_df, asset, 'Send') # crypto "send" _might_ be a disposition (payement, for instance)... of just self-xfer
        send_recs = [r for r in send_df.to_dict(orient='index').values()] 
        wd_df = parse_transactions(main_df, asset, 'Withdrawal') # a cryto "withdrawal" from coinbase (not Pro) _is_ a sale/disposition
        wd_recs = [r for r in wd_df.to_dict(orient='index').values()] 
        
        acqs = [rec_to_acquisition(r, asset) for r in rcv_recs]
        disps = [rec_to_disposition(r, asset) for r in send_recs + wd_recs] # sends _might_ be, crypto WDs *are* disps
    
        data = Stash(asset, f"Coinbase  {asset} txs - {year}", acqs, disps)
        
        #json.dumps(jd)
        with open(filebase+f'-{asset}.json', 'w') as f:
            jd = data.to_json_dict()
            json.dump(jd, f, indent=2)



In [114]:
for yr in ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']:
    process_file(yr, ['BTC', 'ETH'] )  # there are probably other cryto assets, but I dont care at th moment

In [48]:
# work below here

In [10]:
recs = parse_transactions(main_df, "BTC", "Receive")

In [27]:
 # **** Take a multi-row DataFrame and write out a list of dicts keyed by column name
[ i for i in recs.to_dict(orient='index').values()] 

[{'ID': '568156c10db704000100bd74',
  'Timestamp': Timestamp('2015-12-28 15:35:29+0000', tz='UTC'),
  'Transaction Type': 'Receive',
  'Asset': 'BTC',
  'Quantity Transacted': 23.48877188,
  'Price Currency': 'USD',
  'Price at Transaction': 425.01,
  'Subtotal': 9982.96294,
  'Total (inclusive of fees and/or spread)': 9982.96294,
  'Fees and/or Spread': 0.0,
  'Notes': 'Received 23.48877188 BTC from an external account'},
 {'ID': '565df2ba84c069000100ec48',
  'Timestamp': Timestamp('2015-12-01 19:19:22+0000', tz='UTC'),
  'Transaction Type': 'Receive',
  'Asset': 'BTC',
  'Quantity Transacted': 27.76054701,
  'Price Currency': 'USD',
  'Price at Transaction': 363.89,
  'Subtotal': 10101.78545,
  'Total (inclusive of fees and/or spread)': 10101.78545,
  'Fees and/or Spread': 0.0,
  'Notes': 'Received 27.76054701 BTC from an external account'},
 {'ID': '56292be63f3fee00010001ef',
  'Timestamp': Timestamp('2015-10-22 18:33:10+0000', tz='UTC'),
  'Transaction Type': 'Receive',
  'Asset': 

# 

In [95]:
# deal with "receive" records
btc_mask = main_df["Asset"]=='BTC'
eth_mask = main_df["Asset"]=='ETH'
usd_mask = main_df["Asset"]=='USD'

In [96]:
receive_mask = main_df["Transaction Type"]=="Receive"
main_df[receive_mask & btc_mask]
#main_df[receive_mask]

,ID,Timestamp,Transaction Type,Asset,Quantity Transacted,Price Currency,Price at Transaction,Subtotal,Total (inclusive of fees and/or spread),Fees and/or Spread,Notes
3,5862bd944ab22704c07bc02f,2016-12-27 19:14:28+00:00,Receive,BTC,4.964819,USD,935.30,4643.59538,4643.59538,0.0,Received 4.96481918 BTC from an external account
5,5845bb434ab22704c07bc02d,2016-12-05 19:08:51+00:00,Receive,BTC,6.179238,USD,753.16,4653.95471,4653.95471,0.0,Received 6.17923776 BTC from an external account
11,582c92134ab22704c07bc02b,2016-11-16 17:06:27+00:00,Receive,BTC,6.367987,USD,726.09,4623.73183,4623.73183,0.0,Received 6.3679872 BTC from an external account
14,581b622a4ab22704c07bc02a,2016-11-03 16:13:30+00:00,Receive,BTC,9.281033,USD,716.21,6647.16866,6647.16866,0.0,Received 9.28103302 BTC from an external account
18,581173734ab22704c07bc029,2016-10-27 03:24:35+00:00,Receive,BTC,5.000000,USD,689.97,3449.85000,3449.85000,0.0,Received 5 BTC from an external account
26,58091f134ab22704c07bc028,2016-10-20 19:46:27+00:00,Receive,BTC,14.815700,USD,627.61,9298.48157,9298.48157,0.0,Received 14.81570015 BTC from an external account
31,57f4139e0f3ef202e4a61ce9,2016-10-04 20:39:58+00:00,Receive,BTC,0.100000,USD,609.50,60.95000,60.95000,0.0,Received 0.1 BTC from an external account
37,57dc36774ab22704c07bc026,2016-09-16 18:14:15+00:00,Receive,BTC,7.668707,USD,606.27,4649.30675,4649.30675,0.0,Received 7.6687066 BTC from an external account
40,57c84c284ab22704c07bc024,2016-09-01 15:41:28+00:00,Receive,BTC,7.842794,USD,571.06,4478.70585,4478.70585,0.0,Received 7.84279384 BTC from an external account
44,57bb48af4ab22704c07bc023,2016-08-22 18:47:11+00:00,Receive,BTC,7.937104,USD,585.79,4649.47623,4649.47623,0.0,Received 7.93710413 BTC from an external account


In [60]:
wd_mask = main_df["Transaction Type"]=="Withdrawal"
main_df[wd_mask]

,ID,Timestamp,Transaction Type,Asset,Quantity Transacted,Price Currency,Price at Transaction,Subtotal,Total (inclusive of fees and/or spread),Fees and/or Spread,Notes
12,5629314c021a35005e000098,2015-10-22 18:56:12+00:00,Withdrawal,BTC,34.924382,USD,274.709513,9594.06,9500.0,-94.06,Withdrawal to USAA - Jim's savings ****5695


In [42]:
mask = main_df["Transaction Type"]=="Send"
main_df[mask]

,ID,Timestamp,Transaction Type,Asset,Quantity Transacted,Price Currency,Price at Transaction,Subtotal,Total (inclusive of fees and/or spread),Fees and/or Spread,Notes
